# 1 Memastikan tensorflow yang digunakan di Watson Studio dan Google Colab adalah versi di atas 2.0.
# 2 Unduh dataset dan extract file dengan metode unzip.
# 3 Menampung direktori setiap kelas pada direktori train dan validasi ke dalam variabel.
# 4 Pre-processing data dengan image augmentation.
# 5 Mempersiapkan data latih yang akan dipelajari oleh model.
# 6 Membangun arsitektur model dengan CNN.
# 7 Compile dan latih model dengan model.compile dan model.fit hingga mendapatkan akurasi yang diinginkan.
# 8 Prediksi gambar ruangan bersih dan ruangan rapi untuk mengetes model.

In [23]:
 # melakukan ekstraksi pada file zip

In [24]:
import zipfile

In [25]:
local_zip = 'messy-vs-clean-room.zip'

In [26]:
zip_reg = zipfile.ZipFile(local_zip, 'r')

In [27]:
zip_reg.extractall()

In [28]:
zip_reg.close()

In [29]:
base_dir='images'

In [30]:
import os
# mendefinisikan nama direktori untuk data latih
train_dir = os.path.join(base_dir, 'train')
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [31]:
# # mendefinisikan nama direktori untuk data validasi
validation_dir = os.path.join(base_dir, 'val')

In [32]:
os.listdir('images/train')

['messy', 'clean']

In [33]:
os.listdir('images/val')

['messy', 'clean']

In [34]:
# membuat direktori ruangan rapi pada direktori data training
train_clean_dir = os.path.join(train_dir, 'clean')

In [35]:
# membuat direktori ruangan berantakan pada direktori data training
train_messy_dir = os.path.join(train_dir, 'messy')

In [36]:
# membuat direktori ruangan rapi pada direktori data validasi
validation_clean_dir = os.path.join(validation_dir, 'clean')

In [37]:
# membuat direktori ruangan berantakan pada direktori data validasi
validation_messy_dir = os.path.join(validation_dir, 'messy')

In [38]:
# melakukan augmentasi gambar di dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [39]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=20,
                                  horizontal_flip=True,
                                  shear_range=0.2,
                                  fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255,
                                rotation_range=20,
                                horizontal_flip=True,
                                shear_range=0.2,
                                fill_mode='nearest')

In [40]:
# menggunakan objek image data generator untuk mempersiapkan data latih
train_generator = train_datagen.flow_from_directory(train_dir,# direktori data latih
                                                   target_size=(150, 150), #mengubah resolusi seluruh gambar menjadi 150x150 piksel
                                                   batch_size=4,
                                                   # karena klasifikasi terdiri dari 2 class maka menggunakan class mode 'binari'
                                                   class_mode='binary')

validation_generator = test_datagen.flow_from_directory(validation_dir,# direktori data validasi
                                                   target_size=(150, 150), #mengubah resolusi seluruh gambar menjadi 150x150 piksel
                                                   batch_size=4,
                                                   # karena klasifikasi terdiri dari 2 class maka menggunakan class mode 'binari'
                                                   class_mode='binary')

Found 192 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [41]:
import tensorflow as tf
# membuat model CNN
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [42]:
# compile model dengan 'adam' optimize loss function 'binary_crossentropy'
model.compile(loss='binary_crossentropy',
             optimizer='Adam',
             metrics=['accuracy'])

In [44]:
# latih model dengan model.fit
model.fit(train_generator,
          steps_per_epoch=25, # berapa batch yang akan dieksekusi pada setiap epoch
          epochs=20, # tambahkan epoch jika akurasi model belum optimal
          validation_data=validation_generator, # menampilkan akurasi pengujian data validasi
          validation_steps=5, # berapa batch yang akan dieksekusi pada setiap epoch
          verbose=2
         )

Epoch 1/20


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential_1/conv2d_4/Conv2D (defined at <ipython-input-43-a9f5819c460f>:7) ]] [Op:__inference_train_function_2120]

Function call stack:
train_function


In [ ]:
# menggunakan model untuk mengklasifikasikan gambar kamar

import numpy as np 
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

# masukkan file nama gambar
fn = '10.png'
path = fn
img = image.load_img(path, target_size=(150,150))
imgplot = plt.imshow(img)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = model.predict(images, batch_size=10)

print(fn)

if classes==0:
    print('clean')
else:
    print('messy')